## Load Packages

In [19]:
import hashlib
import os
import json
import datetime as date
import pandas as pd

## Creating Helper Functions

In [20]:
def hash_file(filename):
    BLOCKSIZE = 65536
    hasher = hashlib.sha256()
    with open(filename, 'rb') as afile:
        buf = afile.read(BLOCKSIZE)
        while len(buf) > 0:
            hasher.update(buf)
            buf = afile.read(BLOCKSIZE)
    return(hasher.hexdigest())

In [21]:
## maybe do not need the timestamp and name included?

def hash_block(block):
    sha = hashlib.sha256()
    sha.update(str(block['name']) + 
               str(block['timestamp']) + 
               str(block['data']) + 
               str(block['previous_hash']))
    return sha.hexdigest()

## Hash Input File

This can be used as an id of the file.  It is based off of the file contents and therefore if anything in the file changes so does the hash.

In [22]:
hash_file('iris.csv')

'396c921bc9cf625a4ab755540084aa3d0d941c4ffed8681299689b1f502c3ac2'

## Testing Script
This part is a small development portion to be used in the making of a pthon script below.

In [23]:
iris = pd.read_csv('iris.csv')
iris_group = iris.groupby('Species').mean()
iris_group.to_csv('iris_group.csv')

## Creating Script

In [24]:
%%writefile iris_groupmeans.py

import pandas as pd

iris = pd.read_csv('iris.csv')
iris_group = iris.groupby('Species').mean()
iris_group.to_csv('iris_group.csv')

Overwriting iris_groupmeans.py


In [25]:
hash_file('iris_groupmeans.py')

'5f73f6b8ac9602fd25ad45846268d4e74836d1a724cad8ee88ad0a9c859455ac'

In [26]:
hash_file('iris_group.csv')

'fd570681461a0990b120ae70ca2bddd196a908e3c67dd0b5478dbda8afe7d54f'

In [27]:
block = {'name': 'iris_group.csv', 
         'data': hash_file('iris_group.csv'), 
         'timestamp': str(date.datetime.now()), 
         'previous_hash': [hash_file('iris.csv'), hash_file('iris_groupmeans.py')], 
         'hash': ''}

In [28]:
block

{'data': 'fd570681461a0990b120ae70ca2bddd196a908e3c67dd0b5478dbda8afe7d54f',
 'hash': '',
 'name': 'iris_group.csv',
 'previous_hash': ['396c921bc9cf625a4ab755540084aa3d0d941c4ffed8681299689b1f502c3ac2',
  '5f73f6b8ac9602fd25ad45846268d4e74836d1a724cad8ee88ad0a9c859455ac'],
 'timestamp': '2018-09-18 21:10:31.182363'}

In [29]:
block['hash'] = hash_block(block)

In [30]:
block

{'data': 'fd570681461a0990b120ae70ca2bddd196a908e3c67dd0b5478dbda8afe7d54f',
 'hash': 'da164ae8c9b65bca696752b7943fcabac02f3978cd76a98edd0fcb3e44121cae',
 'name': 'iris_group.csv',
 'previous_hash': ['396c921bc9cf625a4ab755540084aa3d0d941c4ffed8681299689b1f502c3ac2',
  '5f73f6b8ac9602fd25ad45846268d4e74836d1a724cad8ee88ad0a9c859455ac'],
 'timestamp': '2018-09-18 21:10:31.182363'}